In [1]:
import os
import shutil
import sys
sys.path.extend(['../../IndoorPathlossRadioMapPrediction/'])

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import imageio.v3 as iio
from matplotlib import pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from PIL import Image

from utils import pad_to_square, draw_radiation_pattern, get_pl_estimation
from algorithm import ICASSP
from networks.vit_pp_upernet import ViTPlusPlusUPerNet

%load_ext autoreload
%autoreload 2

In [2]:
network = ViTPlusPlusUPerNet(
    **{
        "image_size": 518,
        "min_mlp_tokens": 0, 
        "mixer_out": None,
        "mlp_input_dim": 19, 
        "neck_input_dim": 512, 
        "neck_scales": [14, 14, 14, 8, 8, 8, 4, 4, 4, 2, 2, 2, 1, 1], 
        "neck_size": [32, 32, 32, 64, 64, 64, 128, 128, 128, 256, 256, 256, 512, 512], 
        "num_channels": 6, 
        "num_classes": 1, 
        "pre_out_channels": None, 
        "pretrained": "facebook/dinov2-base", 
        "res_hidden_states": None, 
        "up_pool_scales": [1, 2, 4, 7], 
        "use_upernet": True, 
        "v_hidden_size": 768, 
        "v_num_attention_heads": 12, 
        "v_num_channels": 3, 
        "v_num_hidden_layers": 12, 
        "v_patch_size": 14
    }
)
alg_conf = {
    "fixed_scale": False, 
    "out_norm": 160,
    "network": network
}
alg = ICASSP.load_from_checkpoint(
    "/nfs/dgx/raid/iot/outputs/2025-04-01_12-52-21.554391/checkpoints/hard/epoch_0028.ckpt",
    **alg_conf
)
alg.network.eval()

/auto/home/mkrtchyan/miniforge3/envs/wair_d_distal/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ViTPlusPlusUPerNet(
  (conv): Conv2d(6, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (vit_pp): ViTPlusPlus(
    (vit): Dinov2Model(
      (embeddings): Dinov2Embeddings(
        (patch_embeddings): Dinov2PatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): Dinov2Encoder(
        (layer): ModuleList(
          (0-11): 12 x Dinov2Layer(
            (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (attention): Dinov2Attention(
              (attention): Dinov2SelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): Dinov2SelfOutput(
                

In [3]:
def get_input(img, f, rp_csv_path, azimuth, size=518):
    orig_size = img.shape[:2]
    rp_img = draw_radiation_pattern(rp_csv_path, img, azimuth)
    img = np.concatenate([img[..., :4], rp_img[..., np.newaxis]], axis=2)
    pl_init = get_pl_estimation(img, freq=f)
    img = np.concatenate([img, pl_init], axis=-1)
    img = img / np.array([25, 20, 200, 40, 160])
    img = pad_to_square(img)
    img = resize(img, (size, size))
    img = torch.from_numpy(img.astype(np.float32)).permute((2, 0, 1))
    freq_channel = torch.full((1, img.shape[1], img.shape[2]), f, dtype=torch.float32)
    img = torch.cat((img, freq_channel), dim=0)
    img = img.unsqueeze(0)
    return img

def get_pred(pred_image, input_img):
    pred_image = torch.sigmoid(pred_image).squeeze(0).squeeze(0).detach().cpu().numpy()
    mask = np.ones_like(input_img[..., 0])
    mask = pad_to_square(mask, fill_value=0).astype(bool)
    pred_image = resize(pred_image, mask.shape)
    pred_image = pred_image[mask].reshape(input_img[..., 0].shape)
    return pred_image * 160

In [4]:
Buildings = range(1, 7)
ant_ids = [1, 2, 3]
freq = [1, 2]

solution = pd.DataFrame()

for Antenna_ID in (ant_ids):
    for f_i in  freq:
        for b in (Buildings):
            for sp in tqdm(range(0, 80), total=80):
                image_name = "B" + str(b) +  "_Ant"+  str(Antenna_ID) + "_f"  + str(f_i) + "_S" + str(sp)
                img_path = "../ICASSP_TEST_DATA//Inputs/Task_3/" + image_name + '.png'
                if not os.path.exists(img_path):
                    continue
                input_image = iio.imread(img_path)
                rp_csv_path = f"../ICASSP_TEST_DATA/Test_Radiation_Patterns/Ant{Antenna_ID}_Pattern.csv"
                azimuth = pd.read_csv(f"../ICASSP_TEST_DATA/Test_Data_Positions/Positions_B{b}_Ant{Antenna_ID}_f{f_i}.csv").iloc[sp]["Azimuth"]
                your_input_tensor = get_input(input_image, 0.0868 if f_i == 1 else 0.24, rp_csv_path, azimuth)
                out = alg.network(your_input_tensor.to("cuda:0"))
                y_PL = get_pred(out, input_image) # Note that y_PL should have the same dimensions, W x H,  as the input image               
                y = Image.fromarray(y_PL).convert("RGB")
                y.save(f"../preds/task3/{image_name}.png")

100%|████████████████████████████████████████████████████████████████| 80/80 [00:19<00:00,  4.09it/s]
